# Face Expression Detection (Face Detection Using DNN)


In [7]:
import cv2,time
import numpy as np
from tensorflow.keras.models import load_model
# Loading model
newmodel = load_model('FaceExpressionDetection.h5')
expression=['angry','disgust','fear','happy','neutral','sad','surprise'] 
net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt','res10_300x300_ssd_iter_140000.caffemodel')

In [8]:
def facedetect(img):
   
    img = cv2.resize(img,(900,700))
    (h,w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img,(300,300)),1.0,(300,300),(104.0, 177.0, 123.0))
    # print("[INFO] computing object detections...")
    net.setInput(blob)
    detections = net.forward()
    #[,frame,no of detections,[classid,class score,conf,x,y,h,w]]
    # print(detections)
    for i in range(0,detections.shape[2]):
        #extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0,0,i,2]
    #     print(confidence)
        if confidence > 0.5:
            #compute the (x,y) -coordinates of the bounding box for the object
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            (startx,starty,endx,endy)=box.astype("int")
#             text = "{:.2f}%".format(confidence*100)
            
            faceforpredict = img[starty:endy,startx:endx]
            Cvt_gray = cv2.cvtColor(faceforpredict,cv2.COLOR_BGR2GRAY)
#             print(Cvt_gray.shape)
            resize = cv2.resize(Cvt_gray,(48,48))/255.0;
#             print(resize.shape)
            reshaped = np.reshape(resize,(1,48,48,1));
#             print(reshaped.shape)
            prediction = newmodel.predict(reshaped)
#             print(prediction)
            max = np.argmax(prediction)
#             print(max)            
            
            y = starty-10 if starty-10 >10 else starty+10
            cv2.rectangle(img,(startx,starty),(endx,endy),(0,0,255),2)
            cv2.putText(img,expression[max],(startx,y),cv2.FONT_HERSHEY_SIMPLEX,0.45,(0,0,255),2)
    return img

In [9]:
video = cv2.VideoCapture(0)
while True:
    check,frame = video.read()
    img = facedetect(frame)
#     resized_image1 = cv2.resize(img,(900,700))
    cv2.imshow("live",img)
    key = cv2.waitKey(5)
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows()  
